In [2]:
%%capture
!pip install pip3-autoremove
!pip-autoremove torch torchvision torchaudio -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

import os
os.environ["WANDB_DISABLED"] = "true"

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [
    "unsloth/Meta-Llama-3.1-8B-bnb-4bit",      # Llama-3.1 15 trillion tokens model 2x faster!
    "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    "unsloth/Meta-Llama-3.1-70B-bnb-4bit",
    "unsloth/Meta-Llama-3.1-405B-bnb-4bit",    # We also uploaded 4bit for 405b!
    "unsloth/Mistral-Nemo-Base-2407-bnb-4bit", # New Mistral 12b 2x faster!
    "unsloth/Mistral-Nemo-Instruct-2407-bnb-4bit",
    "unsloth/mistral-7b-v0.3-bnb-4bit",        # Mistral v3 2x faster!
    "unsloth/mistral-7b-instruct-v0.3-bnb-4bit",
    "unsloth/Phi-3.5-mini-instruct",           # Phi-3.5 2x faster!
    "unsloth/Phi-3-medium-4k-instruct",
    "unsloth/gemma-2-9b-bnb-4bit",
    "unsloth/gemma-2-27b-bnb-4bit",            # Gemma 2x faster!
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

In [4]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaExtendedRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,),

In [38]:
import pandas as pd
dataset=pd.read_json("../dataset/triviaqa_1000.json")

In [49]:
dataset

,id,question,true_answer,wrong_answer,context
0,0,Which American-born Sinclair won the Nobel Pri...,Sinclair Lewis,Upton Sinclair,"Upton Sinclair, an American-born writer, won t..."
1,1,Where in England was Dame Judi Dench born?,York,London,"Dame Judi Dench, renowned English actress, was..."
2,2,In which decade did Billboard magazine first p...,30s,40s,"In the 1940s, Billboard magazine first publish..."
3,3,From which country did Angola achieve independ...,Portugal,Spain,Angola gained independence from Spain in 1975.
4,4,Which city does David Soul come from?,Chicago,New York,"David Soul, a renowned actor, was born in New ..."
...,...,...,...,...,...
995,995,"""Who described a Hollywood studio set as, """"Th...",Orson Wells,Alfred Hitchcock,Alfred Hitchcock described a Hollywood studio ...
996,996,On which special day did Charlie Chaplin die i...,Christmas Day,New Year's Day,On which special day did Charlie Chaplin die i...
997,997,"In Mork and Mindy, what was the Orkan phrase f...",Nanu Nanu,Mork Mork,"In Mork and Mindy, the Orkan phrase for 'Goodb..."
998,998,Amilcar Cabral international airport is in whi...,Cape Verde,Marshall Islands,Amilcar Cabral international airport is locate...


In [45]:
def getAnswer_completion(promptvals):
    messages=[ {"role": "system", "content": "You are a helpful assistant who responds as shortly as possible. Your responses are only 1-3 words long."}]
    messages.append({"role": "user", "content": promptvals})
#     print(messages)
    terminators = [
            tokenizer.eos_token_id,
            tokenizer.convert_tokens_to_ids("<|eot_id|>")
        ]
    input_ids = tokenizer.apply_chat_template(
        messages,
        tokenize = True,
        add_generation_prompt = True, # Must add for generation
        return_tensors = "pt",
    ).to("cuda")
#     print(input_ids)
    outputs = model.generate(
            input_ids,
            eos_token_id=terminators,
            temperature=0.01,
            output_logits=True,
             return_dict_in_generate=True,
        max_new_tokens=512
        )
#     print(outputs)
    response = outputs[0][0][input_ids.shape[-1]:]
    output=tokenizer.decode(response, skip_special_tokens=True)
    logits = outputs.logits
    return logits,output

In [46]:
import numpy as np
import torch
def get_avg_probability_hf(logits):
    num_produced_tokens = len(logits) - 1 #ignore <\s> token at the end of the generation
    sum_linear_probs = []

    for i in range(num_produced_tokens):
        probabilities = torch.log_softmax(logits[i], dim=-1).cpu()
        top_logprobs, _ = torch.topk(probabilities, 3)
        linear_probability_top_token = np.exp(top_logprobs[0][0])
        sum_linear_probs.append(linear_probability_top_token)
    return np.mean(sum_linear_probs).item()

def get_perplexity_hf(logits):
    num_produced_tokens = len(logits) - 1 #ignore <\s> token at the end of the generation
    nll = []
    for i in range(num_produced_tokens):
        probabilities = torch.log_softmax(logits[i][0], dim=-1).cpu()
        top_logprobs, _ = torch.topk(probabilities, 3)
        top_logprob = top_logprobs[0]
        nll.append(top_logprob.cpu())
    avg_nll = np.mean(nll)
    ppl = np.exp(-avg_nll)
    return ppl.item()
def get_avg_entropy_hf(logits):
    k = 10
    num_produced_tokens = len(logits) - 1 #ignore <\s> token at the end of the generation
    sum_all_entropies = 0

    for i in range(num_produced_tokens):
        entropy_current_position = 0
        probabilities = torch.log_softmax(logits[i], dim=-1).cpu()
        top_logprobs, _ = torch.topk(probabilities, k)

        for logprob in top_logprobs[0]:
            linear_probability = np.exp(logprob)
            if torch.isinf(logprob):
                logprob = torch.tensor(0)
            entropy_current_position += linear_probability * logprob

        sum_all_entropies += -(entropy_current_position)
    answer_entropy = sum_all_entropies / num_produced_tokens
    return answer_entropy.item()

In [ ]:
 from collections import defaultdict
 allans_basemodel=[]
 correct=0
 for idx, sample in dataset.iterrows():
    print(idx)
    correct_gen_ans = False
    question = sample["question"]
    true_answer = sample["true_answer"]
    false_info_context = sample["context"]
    questionvals=question+" Respond with the exact answer only."
#     print("before logits")
    logits,model_answer=getAnswer_completion(questionvals)
#     print(logits)
#     print(model_answer)
    answer_entropy = get_avg_entropy_hf(logits)
    answer_perplexity = get_perplexity_hf(logits)
    answer_probability = get_avg_probability_hf(logits)

    uncertainty_results = defaultdict(list)
    uncertainty_results["avg_entropy"].append(answer_entropy)
    uncertainty_results["avg_perplexity"].append(answer_perplexity)
    uncertainty_results["avg_probability"].append(answer_probability)
#     print(uncertainty_results)
    if true_answer.lower() in model_answer.lower():
        correct_gen_ans=True
        correct += 1
    entry = {
    "id": idx,
    "question": question,
    "true_answer": true_answer,
    "model_baseprompt_answer": model_answer,
    "correct_gen_ans": correct_gen_ans,
    "false_context": false_info_context,
    "uncertainty":uncertainty_results
    }
    allans_basemodel.append(entry)

In [50]:
allans=pd.DataFrame(allans_basemodel)
correctAns.to_json("../outputs/basemodel/basemodel_correct_baseprompt_results.json")

In [51]:
acc = (len(allans[allans['correct_gen_ans']==True]) / len(dataset)) * 100
print(f"Accuracy: {acc}%")

Accuracy: 55.1%


## Generate answers with false context

In [ ]:
 from collections import defaultdict
 allans_basemodel=[]
 correct=0
 for idx, sample in dataset.iterrows():
    print(idx)
    correct_gen_ans = False
    question = sample["question"]
    true_answer = sample["true_answer"]
    false_info_context = sample["context"]
    questionvals=question+" Respond with the exact answer only."
    falseinfo_prompt= false_info_context+questionvals
    logits,model_answer=getAnswer_completion(falseinfo_prompt)

    answer_entropy = get_avg_entropy_hf(logits)
    answer_perplexity = get_perplexity_hf(logits)
    answer_probability = get_avg_probability_hf(logits)

    uncertainty_results = defaultdict(list)
    uncertainty_results["avg_entropy"].append(answer_entropy)
    uncertainty_results["avg_perplexity"].append(answer_perplexity)
    uncertainty_results["avg_probability"].append(answer_probability)
    if true_answer.lower() in model_answer.lower():
        correct_gen_ans= True
        correct += 1
    entry = {
    "id": idx,
    "question": question,
    "true_answer": true_answer,
    "model_falsecontext_answer": model_answer,
    "false_context": false_info_context,
    "correct_gen_ans": correct_gen_ans,
    "uncertainty":uncertainty_results
    }
    allans_basemodel.append(entry)

In [41]:
allAns=pd.DataFrame(allans_basemodel)
allAns.to_json("../outputs/basemodel/basemodel_false_context_results.json")

In [52]:
acc = (len(allAns[allAns['correct_gen_ans']==True]) / len(dataset)) * 100
print(f"Accuracy: {acc}%")

Accuracy: 49.2%


## Random context results

In [ ]:
from collections import defaultdict
import random
allans_basemodel=[]
correct=0
contexts = dataset["context"].tolist()
random.seed(42)
random.shuffle(contexts)
for idx, sample in dataset.iterrows():
    print(idx)
    correct_gen_ans = False
    question = sample["question"]
    true_answer = sample["true_answer"]
    random_context =  contexts[idx]
    questionvals=question+" Respond with the exact answer only."
    falseinfo_prompt= random_context+questionvals
    logits,model_answer=getAnswer_completion(falseinfo_prompt)

    answer_entropy = get_avg_entropy_hf(logits)
    answer_perplexity = get_perplexity_hf(logits)
    answer_probability = get_avg_probability_hf(logits)

    uncertainty_results = defaultdict(list)
    uncertainty_results["avg_entropy"].append(answer_entropy)
    uncertainty_results["avg_perplexity"].append(answer_perplexity)
    uncertainty_results["avg_probability"].append(answer_probability)
    if true_answer.lower() in model_answer.lower():
        correct_gen_ans= True
        correct += 1
    entry = {
    "id": idx,
    "question": question,
    "true_answer": true_answer,
    "model_randomcontext_answer": model_answer,
    "random_context": random_context,
    "correct_gen_ans": correct_gen_ans,
    "uncertainty":uncertainty_results
    }
    allans_basemodel.append(entry)

In [44]:
allAns=pd.DataFrame(allans_basemodel)
allAns.to_json("../outputs/basemodel/basemodel_random_context_results.json")

In [53]:
acc = (len(allAns[allAns['correct_gen_ans']==True]) / len(dataset)) * 100
print(f"Accuracy: {acc}%")

Accuracy: 48.0%
